NOTE: A function definition is a live thing that is a run time object.<br>
In C and C++, you have function pointers but this is really just determined by compiler i.e. a memory addresses of where to put the bag of bits.<br>
In python it's a lot simplier. Every object has some run time life/existence. <b>You can see it in memory and ask it questions.</b>

In [1]:
def add(x, y=10):
    return x + y

In [2]:
add

<function __main__.add(x, y=10)>

In [3]:
add.__name__

'add'

In [4]:
add.__module__

'__main__'

In [5]:
add.__defaults__

(10,)

In [6]:
add.__code__.co_code

b'|\x00|\x01\x17\x00S\x00'

In [11]:
# convert the above bits into the equiavelent opcode.h functions
from dis import dis
dis(add.__code__.co_code)

          0 LOAD_FAST                0 (0)
          2 LOAD_FAST                1 (1)
          4 BINARY_ADD
          6 RETURN_VALUE


In [7]:
add.__code__.co_varnames

('x', 'y')

In [9]:
from inspect import getsource
getsource(add)

'def add(x, y=10):\n    return x + y\n'

#### Profiling a Function

In [14]:
# some library code

def add(x, y=10):
    return x + y

def sub(x, y=10):
    return x - y

In [15]:
from time import time
before = time()
print('add(10)', add(10))
after = time()
print('time taken:', after - before)

add(10) 20
time taken: 0.0004076957702636719


NOTE: If I had multiple functions in my library, I'd need to add the time functionality to all my functions!

#### Remember: Everything has some runtime representation.
Maybe we can write a function called timer and then we wouldn't need to change as much code i.e. simply call the timer function in the user (print statements) code.

In [25]:
from time import time

def timer(func, x, y=10):
    before = time()
    rv = func(x, y)
    after = time()
    print('elapsed', after - before)
    return rv

def add(x, y=10):
    return x + y

print('add(10)', timer(add, 10))
print('add(10)', timer(add, "a", "b"))

elapsed 9.5367431640625e-07
add(10) 20
elapsed 7.152557373046875e-07
add(10) ab


Let's extend this i.e. let's pass in the function and wrap the function.<br>
NOTE: We can define functions at run time. We can define functions anywhere.
<br><br>
And we don't need to change the user code at all.
<br><br>
All we do is create a new function, that takes an existing function and wraps it with a bit more functionality.
<br><br>
This pattern of something calling something = decorator.

In [26]:
from time import time

def timer(func):
    def f(*args, **kwargs): # a wrapper around func
        before = time()
        rv = func(*args, **kwargs)
        after = time()
        print('elapsed', after - before)
        return rv
    return f

<b>Instead of:</b>
```
def add(x, y=10):
    return x + y

add = timer(add)
```
<b>We do this</b>
```
@timer
def add(x, y=10):
    return x + y

print('add(10)', add(10))
```

### Example: Double Decorator

In [29]:
def ntimes(n): # function outside that constructs the decorator (programmatic behaviour)
    def inner(f): # decoractor constructs the wrapper
        def wrapper(*args, **kwargs): # wrapper constructs the function
            for _ in range(n):
                print('running {.__name__}'.format(f))
                rv = f(*args, **kwargs)
            return rv
        return wrapper
    return inner

@ntimes(2)
def add(x, y=10):
    return x + y

In [30]:
add(10)

running add
running add


20